# Employee database

The database for these exercises is `ex_employee1`.

In the employee table, the column `manager_id` contains the _employee id_ of the manager of that employee. If the value is NULL, this means that employee doesn't have a manager.

This is a _small_ dataset. It should be possible to answer these questions by hand, and compare the results to those produced by your SQL query.

If you think you have found an error in the posted results, please create a Github Issue.

In [1]:
%load_ext sql
%sql postgres://localhost/ex_employee1

'Connected: @ex_employee1'

## Questions

1. **List all the employees in order of descreasing salary**

In [2]:
%%sql
SELECT
    *
FROM
    employee
ORDER BY
    salary DESC
;

 * postgres://localhost/ex_employee1
16 rows affected.


empid,name,managerid,deptid,salary
4,Anthony Stark,None,1,100000.0
15,Joe McMillan,13,2,80000.0
8,Shuri,None,5,80000.0
14,Cameron Howe,13,3,75000.0
13,Donna Clarke,None,1,75000.0
9,Diane Rawlinson,None,1,50000.0
11,Nancy Karrigan,9,4,42000.0
16,Saul Goodman,13,6,40000.0
12,Kristi Yamaguchi,9,4,40000.0
10,Tonya Harding,9,4,30000.0


2. **List all the department names, and the number of employees in that department. Order by number of employess in department (greatest to least)**

In [3]:
%%sql
SELECT
    deptname, count(empid)
FROM
    department
    LEFT JOIN
        employee
    ON
        employee.deptid = department.deptid
GROUP BY 
    deptname
ORDER BY 
    count(empid) DESC
;

 * postgres://localhost/ex_employee1
7 rows affected.


deptname,count
Research,4
Admin,4
Training,3
Sales,3
Legal,1
IT,1
SecretOps,0


3. **List all the employees that don't have a manager**

In [4]:
%%sql
SELECT
    *
FROM
    employee
WHERE
    managerid IS NULL
;

 * postgres://localhost/ex_employee1
5 rows affected.


empid,name,managerid,deptid,salary
1,Micheal Scott,None,1,6000.0
4,Anthony Stark,None,1,100000.0
8,Shuri,None,5,80000.0
9,Diane Rawlinson,None,1,50000.0
13,Donna Clarke,None,1,75000.0


4. **List all employees by name, and the name of their manager. If the employee doesn't have a manager, leave the column as NULL.**

In [5]:
%%sql
SELECT
    e.name employee
    , m.name manager
FROM
        employee e
    LEFT JOIN
        employee m
    ON
        e.managerid = m.empid

 * postgres://localhost/ex_employee1
16 rows affected.


employee,manager
Dwight Schrute,Micheal Scott
Jim Halpert,Micheal Scott
Bruce Banner,Anthony Stark
Bill Nye,Anthony Stark
Jane Foster,Anthony Stark
Nancy Karrigan,Diane Rawlinson
Tonya Harding,Diane Rawlinson
Kristi Yamaguchi,Diane Rawlinson
Cameron Howe,Donna Clarke
Saul Goodman,Donna Clarke


5. **For each manager, list the number of employees he or she is managing. For these purposes, a manager is anyone who is not managed by someone else, even if that person has no direct reports.**

In [6]:
%%sql
SELECT
    m.name
    , count(e.empid) as report_count
FROM
        employee m
    LEFT JOIN
        employee e 
    ON
        m.empid = e.managerid
WHERE
    m.managerid IS NULL
GROUP BY 
    m.empid, m.name
ORDER BY 
    count(e.empid) DESC
;

 * postgres://localhost/ex_employee1
5 rows affected.


name,report_count
Diane Rawlinson,3
Donna Clarke,3
Anthony Stark,3
Micheal Scott,2
Shuri,0


6. **Find the two highest paid people per department**

Note: This is a good use case for _window functions_, although you can also do it with a where clause.

In [7]:
%%sql
WITH emp_rank AS (
    SELECT
        empid
        ,RANK()
            OVER (PARTITION BY deptid ORDER BY salary DESC)
            AS deptrank
    FROM
        employee
)
SELECT
    e.name
    ,e.empid
    ,e.salary
    ,d.deptid
    ,d.deptname
    ,r.deptrank
FROM
        employee e
    JOIN
        department d
    ON
        d.deptid = e.deptid
    JOIN
        emp_rank r
    ON
        r.empid = e.empid
WHERE
    r.deptrank <= 2
ORDER BY 
    d.deptid ASC, e.salary DESC

 * postgres://localhost/ex_employee1
10 rows affected.


name,empid,salary,deptid,deptname,deptrank
Anthony Stark,4,100000.0,1,Admin,1
Donna Clarke,13,75000.0,1,Admin,2
Joe McMillan,15,80000.0,2,Sales,1
Dwight Schrute,2,4000.0,2,Sales,2
Cameron Howe,14,75000.0,3,IT,1
Nancy Karrigan,11,42000.0,4,Training,1
Kristi Yamaguchi,12,40000.0,4,Training,2
Shuri,8,80000.0,5,Research,1
Bruce Banner,6,8000.0,5,Research,2
Saul Goodman,16,40000.0,6,Legal,1
